In [26]:
from pyspark.sql import SparkSession, functions as F

In [2]:
# Create Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/25 15:06:57 WARN Utils: Your hostname, eli-mac.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/03/25 15:06:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 15:06:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [30]:
# Read in green taxi data for all years and months
green_df = spark.read.parquet('../data/pq/green/*/*')

In [31]:
# Read in yellow taxi data for all years and months
yellow_df = spark.read.parquet('../data/pq/yellow/*/*')

In [32]:
# Rename datetime columns 
green_df = green_df \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [33]:
yellow_df = yellow_df \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [34]:
# Get intersection of columns with order preserved
common_cols = []
green_cols = set(green_df.columns)
for col in yellow_df.columns:
    if col in green_cols:
        common_cols.append(col)

In [35]:
common_cols

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [36]:
# Select common columns from both datasets 
# with an additional column capturing the service type / taxi color
green_df_sel = green_df \
    .select(common_cols) \
    .withColumn('service_type', F.lit('green'))

In [37]:
yellow_df_sel = yellow_df \
    .select(common_cols) \
    .withColumn('service_type', F.lit('yellow'))

In [38]:
# Join / union all green and yellow datasets
trips_df = green_df_sel.unionAll(yellow_df_sel)

In [39]:
# Demo successful union
trips_df.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [41]:
# Tell Spark to treat DataFrame as table for SQL querying
trips_df.createOrReplaceTempView('trips')

In [46]:
# Write SQL to reproduce count of group by on service type done above
spark.sql("""
SELECT 
          service_type,
          COUNT(1)
FROM 
          trips
GROUP BY
          service_type;
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [61]:
# Modified version of dm_monthly_zone_revenue SQL query from module 4
res_df = spark.sql(
     """
     SELECT 
          -- Reveneue grouping 
          PULocationID AS revenue_zone,
          date_trunc('month', pickup_datetime) AS revenue_month, 
          service_type, 

          -- Revenue calculation 
          SUM(fare_amount) AS revenue_monthly_fare,
          SUM(extra) AS revenue_monthly_extra,
          SUM(mta_tax) AS revenue_monthly_mta_tax,
          SUM(tip_amount) AS revenue_monthly_tip_amount,
          SUM(tolls_amount) AS revenue_monthly_tolls_amount,
          SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
          SUM(total_amount) AS revenue_monthly_total_amount,
          
          -- Additional calculations
          AVG(passenger_count) AS avg_monthly_passenger_count,
          AVG(trip_distance) AS avg_monthly_trip_distance
     FROM 
          trips
     GROUP BY
          revenue_zone, revenue_month, service_type
     """
)

In [63]:
# Write resulting DataFrame to parquet with one partition
res_df.coalesce(1).write.parquet('../data/report/revenue/', mode='overwrite')